In [19]:
import sys
import numpy as np
import scipy
import subprocess
from natsort import natsorted
sys.path.append("./utils")
import os
from preprocessing import *
from InterfaceDeclaration import LPBFInterface

alphabet = list(string.ascii_lowercase)

project_name = ["MuSIC", "MaPS", "MuSIC_EXP1"]
if os.name == "posix":
    data_dir = subprocess.getoutput("echo $DATADIR")
elif os.name == "nt":
    data_dir = subprocess.getoutput("echo %datadir%")
music_dir = os.path.join(data_dir, "MuSIC")
if not os.path.exists(music_dir):
    project_name[0] = "2024-MUSIC"
daq_dir = os.path.join(data_dir, *project_name, "Acoustic Monitoring")
lmq_dir = os.path.join(data_dir, *project_name, "LMQ Monitoring")
del music_dir

sampling_rate_daq: int = int(1.25 * 1e6)
sampling_rate_lmq: int = int(0.1 * 1e6)
tdms_daq_list = natsorted(
    [i for i in os.listdir(daq_dir) if i.split(".")[-1] == "tdms"]
)
bin_lmq_list = natsorted([i for i in os.listdir(lmq_dir) if i.split(".")[-1] == "bin"])
lmq_channel_name = [
    "Vector ID",
    "meltpooldiode",
    "X Coordinate",
    "Y Coordinate",
    "Laser power",
    "Spare",
    "Laser diode",
    "Varioscan(focal length)",
]
process_regime = [
    [0,    59, "Base"  ], 
    [60,  129, "KH"    ], 
    [130, 199, "Normal"], 
    [200, 269, "RLoF"  ], 
    [269, 339, "LoF"   ] 
]
layer_i = 129
tdms_file_path =os.path.join(daq_dir,tdms_daq_list[layer_i])
bin_file_path = os.path.join(lmq_dir,bin_lmq_list[layer_i])
data_daq = TdmsFile(tdms_file_path)
data_lmq = read_bin_file(bin_file_path)
_bin_name = os.path.split(bin_file_path)[-1].split(".")[0]

_mic = (data_daq.groups()[0]).channels()[0].data
_ae = (data_daq.groups()[0]).channels()[1].data
_photodiode = (data_daq.groups()[0]).channels()[2].data
_laserdiode = data_lmq[:,6]
_laserpower = data_lmq[:,4]
_daq_t=np.linspace(0,len(_mic)/sampling_rate_daq,len(_mic))
_lmq_t=np.linspace(0,len(_laserdiode)/sampling_rate_lmq,len(_laserdiode))

sliced_indices_daq = np.asarray(pulse_signal_slicer_by_interval( _photodiode, sampling_rate_daq, pulse_on_thershold=0.1, index_only=True))
sliced_indices_lmq = np.asarray(pulse_signal_slicer_by_interval( _laserdiode, sampling_rate_lmq, pulse_on_thershold=0.1, index_only=True))

sliced_indices_lmq_0 = np.zeros([5,2],dtype=int)
for i, vid in enumerate(np.unique(data_lmq[:,0])[1:-1]):
    sliced_indices_lmq_0[i,0], sliced_indices_lmq_0[i,1] = find_first_and_last_indices(data_lmq[:,0], vid)

In [24]:
current_cube_lmq = sliced_indices_lmq[0]
current_cube_daq = sliced_indices_daq[0]

pulse_on_thershold = 0.5
line_indices_lmq = np.asarray(pulse_signal_slicer_by_interval(_laserdiode[current_cube_lmq[0]:current_cube_lmq[1]],sampling_rate=sampling_rate_lmq,min_interval_time=0, index_only=True,pulse_on_thershold=pulse_on_thershold))
line_indices_daq = np.asarray(pulse_signal_slicer_by_interval(_photodiode[current_cube_daq[0]:current_cube_daq[1]],sampling_rate=sampling_rate_lmq,min_interval_time=0, index_only=True,pulse_on_thershold=pulse_on_thershold))
line_indices_lmq.shape,line_indices_daq.shape
while len(line_indices_lmq)!=len(line_indices_daq):
    pulse_on_thershold -= 0.1
    line_indices_lmq = np.asarray(pulse_signal_slicer_by_interval(_laserdiode[current_cube_lmq[0]:current_cube_lmq[1]],sampling_rate=sampling_rate_lmq,min_interval_time=0, index_only=True,pulse_on_thershold=pulse_on_thershold))
    line_indices_daq = np.asarray(pulse_signal_slicer_by_interval(_photodiode[current_cube_daq[0]:current_cube_daq[1]],sampling_rate=sampling_rate_lmq,min_interval_time=0, index_only=True,pulse_on_thershold=pulse_on_thershold))
    